<a href="https://colab.research.google.com/github/fjadidi2001/Machine_Learning_Journey/blob/main/Predict_ATP_tennis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> ATP stands for the Association of Tennis Professionals. It is the governing body for men's professional tennis worldwide. Founded in 1972, the ATP is responsible for organizing the ATP Tour, which includes a series of tournaments such as the ATP Masters 1000, ATP 500, and ATP 250 events, along with the prestigious ATP Finals and the Davis Cup.

> The ATP also ranks male players based on their performance in tournaments, creating the ATP Rankings, which reflect players' results over the previous 52 weeks. The ATP plays a significant role in promoting the sport, securing player rights, and managing tournament standards.




# Problem Statement:
> “Analyzing and Predicting Player Performance in Professional Tennis: An Exploration of Court-Type Specialization and Peak Performance Age”





## Project Objectives
1. **Exploratory Analysis of Player Characteristics**  
   - Analyze the distribution of player ages, rankings, and Elo ratings across different surfaces (hard court, clay court, grass court).
   - Identify trends in age and performance metrics to determine how they vary across court types.

2. **Court-Type Specialization Prediction**  
   - Build a classification model to predict a player's specialization (e.g., "hard-court specialist," "clay-court specialist," or "all-rounder") based on historical performance data.

3. **Peak Performance Prediction**  
   - Investigate the relationship between age and peak performance, specifically focusing on the "Peak Age" and "Peak Elo" features.
   - Use regression techniques to predict a player’s peak Elo rating and the age at which they will reach peak performance.

4. **Gender-Based Performance Analysis**  
   - Conduct a comparative analysis of performance metrics between male and female players, examining if and how gender influences court-type specialization and peak performance.

5. **Evaluation and Interpretability**  
   - Evaluate model performance using metrics such as accuracy, F1-score, mean squared error (MSE), etc., depending on the task (classification or regression).
   - Incorporate interpretability techniques (e.g., SHAP values or feature importance) to provide insights into the factors contributing to a player’s peak performance and specialization.




### Contribution and Impact
This project could provide valuable insights into tennis player development and training, helping coaches and analysts better understand the factors that contribute to a player’s success on different court types. It could also provide insights into the ideal age for peak performance, which is valuable for athlete management and career planning.

# Step 1: load the dataset

In [38]:

# Download latest version
path = kagglehub.dataset_download("anupangadi/tennis-players-ranks-prediction-using-atp-elo")

print("Path to dataset files:", path)




Path to dataset files: /root/.cache/kagglehub/datasets/anupangadi/tennis-players-ranks-prediction-using-atp-elo/versions/1


# Step 2: EDA(Exploratory Data Analysis)

# Step 3: Preprocessing

# Step 4: Split dataset

# Step 5: Train Model

# step 6: Evaluate the model utilizing multiple metrics